# Recurrent Neural Network for Time Series Forecasting

https://www.tensorflow.org/tutorials/structured_data/time_series

In [3]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd


import torch.optim as optim
#from torch.utils.tensorboard import SummaryWriter

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, Theta
from darts.metrics import mape, rmse, r2_score
from darts.utils.statistics import check_seasonality, plot_acf
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import sys
import time
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

In [39]:
# Drop first column of dataframe
def drop_first_col(df):
    return df.iloc[: , 1:]

df = pd.read_csv("dataset\\full.csv")
df = drop_first_col(df)

train = df.loc[0:df.shape[0]*0.8]
test = df.loc[df.shape[0]*0.8:df.shape[0]]

df.set_index('VISCODE',inplace=True)
train.set_index('VISCODE',inplace=True)
test.set_index('VISCODE',inplace=True)

In [40]:
EPOCH = 3         # for testing or debugging, rather than real forecasts, 
                  # set EPOCH to a low value like 3; for the real deal: 300 
                  # 300 will take as much as 30 - 50 minutes of processing time
        
is_seasonal, periodicity = check_seasonality(df, max_lag=240)

# print(type(model_train))
    
# # normalize the time series
# trf = Scaler()
# # fit the transformer to the training dataset
# train_trf = trf.fit_transform(np.squeeze(model_train))
# # apply the transformer to the validation set and the complete series 
# val_trf = trf.transform(model_validation)
# ts_trf = trf.transform(model_test)

AttributeError: 'DataFrame' object has no attribute '_assert_univariate'

In [43]:
# set up, fit, run, plot, and evaluate the RNN model

def run_RNN(flavor, ts, train, val):

    # set the model up
    model_RNN = RNNModel(
        model=flavor,
        model_name=flavor + str(" RNN"),
        input_chunk_length=5,
        training_length=20,
        hidden_dim=20,
        batch_size=16,
        n_epochs=EPOCH,
        dropout=0,
        optimizer_kwargs={'lr': 1e-3},
        log_tensorboard=True,
        random_state=42,
        force_reset=True)
        
    if flavor == "RNN": flavor = "Vanilla" 

    # fit the model
    fit_it(model_RNN, train, val, flavor)

    # compute N predictions
    pred = model_RNN.predict(n=FC_N, future_covariates=covariates) 

    # plot predictions vs actual
    plot_fitted(pred, ts, flavor)

    # print accuracy metrics
    res_acc = accuracy_metrics(pred, ts)    
    print(flavor + " : ")
    _ = [print(k,":",f'{v:.4f}') for k,v in res_acc.items()]

    return [pred, res_acc]

In [45]:

# helper function: fit the RNN model

def fit_it(model, train, val, flavor):
    t_start =  time.perf_counter()
    print("\nbeginning the training of the {0} RNN:".format(flavor))

    res = model.fit(train,
             future_covariates=covariates,
             val_series=val,
             val_future_covariates=covariates,
             verbose=True)

    res_time = time.perf_counter() - t_start
    print("training of the {0} RNN has completed:".format(flavor), f'{res_time:.2f} sec')

    return res


In [46]:
# run 3 different flavors of RNN on the time series:
flavors = ["LSTM", "GRU", "RNN"]

# call the RNN model setup for each of the 3 RNN flavors
res_flavors = [run_RNN(flv, test, train, validation) for flv in flavors]


beginning the training of the LSTM RNN:


NameError: name 'covariates' is not defined